In [1]:
import pandas as pd
import re
import sqlite3
import cufflinks as cf
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
# Configuration
home_path = "./"
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# Loading data

In [6]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect(home_path+"collectors/data.sqlite3")
df = pd.read_sql_query("SELECT * from job_post", con)
con.close()
df.head(5)

,id,title,company,location,description,source,search_kw
0,1,Data Scientist,Aquatic Informatics,"Vancouver, BC",Do you want a meaningful role in a company tha...,indeed.com,data scientist
1,2,Business Intelligence Analyst,GLENTEL,"Burnaby, BC",Brand: Glentel Corporate\nLocation: Burnaby Of...,indeed.com,data scientist
2,3,Human Resources Data Scientist,Rio Tinto,Canada,2 x newly created Data Scientist opportunities...,indeed.com,data scientist
3,4,Lead - Human Resource Data Scientist,Rio Tinto,Canada,Newly created data science lead embedded withi...,indeed.com,data scientist
4,5,Machine Learning Engineer,Skycope Technologies Inc,"Vancouver, BC","Who We are\nFounded in 2016, Skycope Technolog...",indeed.com,data scientist


# Utilities

This section composes of all common functions for data analysis tasks

In [7]:
# Extract top n words using CountVectorizer
def get_top_n_words(corpus, stop_words=None, n=None):
    vec = CountVectorizer(stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

# Extract top n bigrams 
def get_top_n_bigram(corpus, stop_words=None, n=None):
    vec = CountVectorizer(ngram_range=(2, 2), stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

# Extract top n bigrams 
def get_top_n_trigram(corpus, stop_words=None, n=None):
    vec = CountVectorizer(ngram_range=(3, 3), stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

# Job Description Analysis

## Find top words in job description before removing stop words

In [8]:
n_top = 20
common_words = get_top_n_words(df['description'], stop_words=None, n=n_top)
df1 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df1.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Red', title='Top 20 words in job-description before removing stop words')

## Find top words in job description after removing stop words

In [9]:
common_words = get_top_n_words(df['description'], stop_words='english', n=n_top)
df2 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df2.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Green', title='Top 20 words in job-description after removing stop words')

## Find top words in job description of data scientist role

In [10]:
common_words = get_top_n_words(df[df['search_kw'] == 'data scientist']['description'], stop_words='english', n=n_top)
df2 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df2.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Green', title='Top 20 words in job-description of data scientist role')

## Find top words in job description of data engineer role

In [11]:
n_top = 20
common_words = get_top_n_words(df[df['search_kw'] == 'data engineer']['description'], stop_words='english', n=n_top)
df2 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df2.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Green', title='Top 20 words in job-description of data engineer role')

## Find top words in job description of data analyst role

In [12]:
n_top = 20
common_words = get_top_n_words(df[df['search_kw'] == 'data analyst']['description'], stop_words='english', n=n_top)
df2 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df2.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Green', title='Top 20 words in job-description of data analyst role')

## Find top bigrams in job description before removing stop words

In [13]:
common_words = get_top_n_bigram(df['description'], stop_words=None, n=n_top)
df1 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df1.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Red', title='Top 20 bigrams in job-description before removing stop words')

## Find top bigrams in job description after removing stop words

In [14]:
common_words = get_top_n_bigram(df['description'], stop_words='english', n=n_top)
df4 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df4.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Green', title='Top 20 bigrams in the job-description after removing stop words')

## Find top bigrams in job description of data scientist role

In [15]:
common_words = get_top_n_bigram(df[df['search_kw'] == 'data scientist']['description'], stop_words='english', n=n_top)
df1 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df1.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Green', title='Top 20 bigrams in the job-description of data scientist role')

## Find top bigrams in job description of data engineer role

In [16]:
common_words = get_top_n_bigram(df[df['search_kw'] == 'data engineer']['description'], stop_words='english', n=n_top)
df2 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df2.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Green', title='Top 20 bigrams in the job-description of data engineer role')

## Find top bigrams in job description of data analyst role

In [17]:
common_words = get_top_n_bigram(df[df['search_kw'] == 'data analyst']['description'], stop_words='english', n=n_top)
df3 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df3.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Green', title='Top 20 bigrams in the job-description of data analyst role')

## Save all top bigrams 

In [18]:
df1['search_kw'] = 'data scientist'
df2['search_kw'] = 'data engineer'
df3['search_kw'] = 'data analyst'
result = pd.concat([df1, df2, df3])
# result['description'].unique()
result.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Green', title='Top bigrams in the job-description of data scientist, engineer and analyst role')
result.to_csv('results/top_bigrams.csv', index=False)

## Find top trigrams in job description before removing stop words

In [19]:
common_words = get_top_n_trigram(df['description'], stop_words=None, n=n_top)
df1 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df1.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Red', title='Top 20 trigrams in job-description before removing stop words')

## Find top trigrams in job description after removing stop words

In [20]:
common_words = get_top_n_trigram(df['description'], stop_words='english', n=n_top)
df4 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df4.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Green', title='Top 20 trigrams in the job-description after removing stop words')

## Find top trigrams in the job description of data scientist role

In [21]:
common_words = get_top_n_trigram(df[df['search_kw']=='data scientist']['description'], stop_words='english', n=n_top)
df4 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df4.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Green', title='Top 20 trigrams in the job-description of data scientist role')

## Find top trigrams in the job description of data engineer role

In [23]:
common_words = get_top_n_trigram(df[df['search_kw']=='data engineer']['description'], stop_words='english', n=n_top)
df4 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df4.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Green', title='Top 20 trigrams in the job-description of data engineer role')

## Find top trigrams in the job description of data analyst role

In [24]:
common_words = get_top_n_trigram(df[df['search_kw']=='data analyst']['description'], stop_words='english', n=n_top)
df4 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df4.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Green', title='Top 20 trigrams in the job-description of data analyst role')